In [12]:
course_list= ["a"]
course_list.remove("a")
print(course_list)
print(len(course_list))

[]
0


In [31]:

print([course for course in ["a","","   "] if not course.isspace() and course != ""])

['a']


In [28]:
course=""
a = if not course.isspace() and course not == ""
print

SyntaxError: invalid syntax (1086604014.py, line 2)